In [79]:
import time 
import logging
import requests
import re
import sys
from bs4 import BeautifulSoup
from queue import Queue
from urllib import parse, request
from urllib.parse import urlparse
import heapq
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.by import By

In [80]:
site = "https://ocw.mit.edu/search/?d=Electrical%20Engineering%20and%20Computer%20Science"
page = requests.get(site)
soup = BeautifulSoup(page.content, "html.parser")

In [81]:
# Create a webdriver instance
driver = webdriver.Chrome()

# Get the main page with the links to the courses
driver.get('https://ocw.mit.edu/search/?d=Electrical%20Engineering%20and%20Computer%20Science&q=computer%20science')

In [82]:
# Scroll to the bottom of the page to load more content
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for new content to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [83]:
# Find all links to courses
cards = driver.find_elements(By.CLASS_NAME, 'lr-row.course-title')

# Extract links from each card
links = []
for card in cards:
    link_element = card.find_element(By.TAG_NAME, 'a')
    link = link_element.get_attribute("href")
    links.append(link)

#print(links)

['https://ocw.mit.edu/courses/6-042j-mathematics-for-computer-science-fall-2010/', 'https://ocw.mit.edu/courses/6-042j-mathematics-for-computer-science-fall-2005/', 'https://ocw.mit.edu/courses/6-042j-mathematics-for-computer-science-spring-2015/', 'https://ocw.mit.edu/courses/9-66j-computational-cognitive-science-fall-2004/', 'https://ocw.mit.edu/courses/6-080-great-ideas-in-theoretical-computer-science-spring-2008/', 'https://ocw.mit.edu/courses/6-00-introduction-to-computer-science-and-programming-fall-2008/', 'https://ocw.mit.edu/courses/6-00sc-introduction-to-computer-science-and-programming-spring-2011/', 'https://ocw.mit.edu/courses/6-0001-introduction-to-computer-science-and-programming-in-python-fall-2016/', 'https://ocw.mit.edu/courses/6-01sc-introduction-to-electrical-engineering-and-computer-science-i-spring-2011/', 'https://ocw.mit.edu/courses/6-0002-introduction-to-computational-thinking-and-data-science-fall-2016/', 'https://ocw.mit.edu/courses/6-047-computational-biolog

In [84]:
# Visit each link and extract the title and description
for link in links:
    driver.get(link)
    time.sleep(2)  # Add a delay to allow the page to load
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract title
    title_element = soup.find('a', class_='text-capitalize m-0 text-white')
    if title_element:
        title = title_element.text.strip()
    else:
        title = 'Title not found'
    
    # Extract description
    description_element = soup.find('div', id='expanded-description')
    if description_element:
        description = description_element.text.strip()
    else:
        description_element = soup.find('div', id='full-description')
        if description_element:
            description = description_element.text.strip()
        else:
            description = 'Description not found'
    
    print(f"Title: {title}")
    print(f"Description: {description}")
    print()

# Quit the webdriver
driver.quit()


Title: Mathematics for Computer Science
Description: This course covers elementary discrete mathematics for computer science and engineering. It emphasizes mathematical definitions and proofs as well as applicable methods. Topics include formal logic notation, proof methods; induction, well-ordering; sets, relations; elementary graph theory; integer congruences; asymptotic notation and growth of functions; permutations and combinations, counting principles; discrete probability. Further selected topics may also be covered, such as recursive definition and structural induction; state machines and invariants; recurrences; generating functions.Show less

Title: Mathematics for Computer Science
Description: This is an introductory course in Discrete Mathematics oriented toward Computer Science and Engineering. The course divides roughly into thirds:

Fundamental Concepts of Mathematics: Definitions, Proofs, Sets, Functions, Relations
Discrete Structures: Modular Arithmetic, Graphs, State M

KeyboardInterrupt: 

In [86]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer

def extract_keywords(course_description):
    # Tokenize the text into words
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(course_description.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Calculate word frequency
    freq_dist = FreqDist(words)

    # Get the most common 5 words
    keywords = freq_dist.most_common(5)

    return [keyword[0] for keyword in keywords]

# Example course description
course_description = """
This course covers elementary discrete mathematics for computer science and engineering. 
It emphasizes mathematical definitions and proofs as well as applicable methods. Topics include formal logic notation, 
proof methods; induction, well-ordering; sets, relations; elementary graph theory; integer congruences; asymptotic notation and growth of functions; 
permutations and combinations, counting principles; discrete probability. Further selected topics may also be covered, such as recursive definition and structural induction; 
state machines and invariants; recurrences; generating functions.
"""

# Extract keywords from the course description
keywords = extract_keywords(course_description)
print(keywords)

['elementary', 'discrete', 'well', 'methods', 'topics']


In [6]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained sentence transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example course description
course_description = """
This course covers elementary discrete mathematics for computer science and engineering. 
It emphasizes mathematical definitions and proofs as well as applicable methods. Topics include formal logic notation, 
proof methods; induction, well-ordering; sets, relations; elementary graph theory; integer congruences; asymptotic notation and growth of functions; 
permutations and combinations, counting principles; discrete probability. Further selected topics may also be covered, such as recursive definition and structural induction; 
state machines and invariants; recurrences; generating functions.
"""

# User input
user_goals = input("What are your goals for an online CS course? ")

# Encode the user's goals and the course description
user_goals_embedding = model.encode(user_goals, convert_to_tensor=True)
course_description_embedding = model.encode(course_description, convert_to_tensor=True)

# Calculate the similarity between the embeddings
cosine_score = util.pytorch_cos_sim(user_goals_embedding, course_description_embedding).item()

print(f"Similarity Score: {cosine_score}")


What are your goals for an online CS course?  This course covers elementary discrete mathematics for computer science and engineering.  It emphasizes mathematical definitions and proofs as well as applicable methods. Topics include formal logic notation,  proof methods; induction, well-ordering; sets, relations; elementary graph theory; integer congruences; asymptotic notation and growth of functions;  permutations and combinations, counting principles; discrete probability. Further selected topics may also be covered, such as recursive definition and structural induction;  state machines and invariants; recurrences; generating functions.


Similarity Score: 1.0000001192092896


#### NOTES

TITLE: <a class="text-capitalize m-0 text-white" href="/courses/6-001-structure-and-interpretation-of-computer-programs-spring-2005/">Structure and Interpretation of Computer Programs</a>

DESCR: This course introduces students to the principles of computation. Upon completion of 6.001, students should be able to explain and apply the basic methods from programming languages to analyze computational systems, and to generate computational solutions to abstract problems. Substantial weekly programming assignments …

SYLLABUS_URL: